In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Obtaining dependency information for crewai==0.28.8 from https://files.pythonhosted.org/packages/d2/78/dd949ea0d914ce306821067a886b3520755b59df71cd868a2997cc90a7a5/crewai-0.28.8-py3-none-any.whl.metadata
  Obtaining dependency information for crewai_tools==0.1.6 from https://files.pythonhosted.org/packages/de/35/7b0ab9c9de202d2fc22dc183ec9da28c992038938f1e06684f72ba15eb55/crewai_tools-0.1.6-py3-none-any.whl.metadata
  Obtaining dependency information for langchain_community==0.0.29 from https://files.pythonhosted.org/packages/e1/80/4136757bf245f60e2f4e148adb340207ea330be92bc26727bca32fe6bc48/langchain_community-0.0.29-py3-none-any.whl.metadata
  Obtaining dependency information for instructor<0.6.0,>=0.5.2 from https://files.pythonhosted.org/packages/3d/57/35e22db00463ac2a2f57e7b8823d0084c5e83f69d518313bdfbf61764463/instructor-0.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for langchain<0.2.0,>=0.1.10 from https://files.pythonhosted.org/packages/4b/28/da40a6b12e78

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [8]:
# utils.py or a separate helper file
import os
from dotenv import load_dotenv

def get_openai_api_key():
    load_dotenv()  # Load environment variables from .env
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY is not set. Please check your .env file.")
    return api_key


In [9]:
import os
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

Creating Agents
- Define your Agents, and provide them a role, goal and backstory.
- It has been seen that LLMs perform better when they are role playing.

In [10]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### AGENT WRITER

In [12]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### AGENT EDITOR

In [13]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

Creating Tasks
- Define your Tasks, and provide them a description, expected_output and agent.
Task: Plan

Task: Plan

In [15]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

TASK: WRITE

In [16]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

TASK : EDIT

In [17]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

Creating the Crew
- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
- verbose=2 allows you to see all the logs of the execution.

In [18]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

Running the Crew

In [19]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document

Title: Unleashing the Power of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries.
- Hook to grab the audience's attention and introduce the main topic of the article.

Key Points:
1. Latest Trends in Artificial Intelligence:
- Advancements in machine learning algorithms.
- Integration of AI in healthcare, finance, and retail sectors.
- Rise of natural language processing and chatbot

In [20]:
from IPython.display import Markdown
Markdown(result)

# Unleashing the Power of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has revolutionized various industries, shaping the way we live and work. From advancements in machine learning algorithms to the integration of AI in healthcare, finance, and retail sectors, the impact of AI is undeniable. As we delve into the latest trends and key players in the AI industry, it becomes evident that AI is not just a buzzword but a powerful tool driving innovation and transformation across different domains.

## Latest Trends in Artificial Intelligence

One of the most significant trends in AI is the continuous advancements in machine learning algorithms. With the development of deep learning models and neural networks, AI systems are becoming more sophisticated and capable of handling complex tasks with precision. Moreover, the integration of AI in healthcare, finance, and retail sectors has opened up new possibilities for personalized services, efficient operations, and enhanced customer experiences. The rise of natural language processing and chatbots has also revolutionized the way we interact with technology, enabling seamless communication and automation in various applications.

## Key Players in the AI Industry

In the competitive landscape of the AI industry, several key players stand out for their groundbreaking research and innovative solutions. The Google Brain team, known for its work in deep learning and AI research, has been at the forefront of driving advancements in the field. IBM Watson, with its cognitive computing capabilities, has paved the way for AI-powered solutions in healthcare, finance, and other industries. OpenAI, a non-profit organization dedicated to AI research, has been instrumental in promoting ethical AI development and ensuring the responsible use of AI technologies. Microsoft Research AI and Amazon Web Services AI are also key players in the industry, offering a wide range of AI tools and services to businesses and developers.

## Noteworthy News on Artificial Intelligence

Keeping up with the latest news on artificial intelligence is crucial for understanding the evolving landscape of AI and its impact on society. Recent breakthroughs in AI research, such as the development of self-learning algorithms and reinforcement learning techniques, have pushed the boundaries of what AI can achieve. However, the impact of AI on job markets and the ethical considerations in AI development raise important questions about the future of work and the responsible use of AI technologies. As we navigate these complex issues, it is essential to stay informed and actively engage in discussions on the ethical and social implications of AI.

## Conclusion

In conclusion, Artificial Intelligence continues to shape the future of technology and society, with the latest trends and key players driving innovation and transformation across various industries. By staying updated on the latest developments in AI, professionals in tech-related fields, business owners, students, and researchers can harness the power of AI to drive growth and create positive impact in their respective fields. As we embrace the potential of AI, it is important to consider the ethical implications and societal challenges that come with it, ensuring that AI technologies are developed and deployed responsibly. Subscribe to our blog for more updates on AI trends, share this article on social media to spread awareness, and leave your thoughts in the comments below. Let's continue the conversation on Artificial Intelligence and unleash its full potential for the benefit of all.